In [ ]:
model_checpoint='distilbert-base-uncased'
## define label maps

In [ ]:
id2label={0:"Negative",1:"Positive"}
label2id={"Negative":0,"Positive":1}


In [ ]:
## Generating classification model from model_checkpoints

In [ ]:
model=AutoModelForSequenceClassification.from_pretrained(model_checpoint,num_labels=2,id2label=id2label,label2id=label2id)


In [ ]:
## Load dataset

In [ ]:
dataset=load_dataset("shawnn/imdb-trained")


In [ ]:
## Preprocess data

In [ ]:
tokenizer=AutoTokenizer.from_pretrained(model_checpoint,add_prefix_space=True)


In [ ]:
## create tokenize function

In [ ]:
def tokenize_function(example):
    text=example(text)
    tokenizer.truncation_side="left"
    tokenized_input=tokenizer(text,return_tensors="np",truncation=True,max_length=512)
    return tokenized_input


In [ ]:
## create data collator

In [ ]:
data_collator=DataCollatorWithPadding(tokenizer=tokenizer)


In [ ]:
## evaluation metrics

In [ ]:
accuracy=evaluate.load("accuracy")

## defining an evaluating function to pass into trainer later


def compute_metrics(p):
    predictions,labels=p
    predictions=np.argmax(predictions,axis=1)
    return {"accuracy":accuracy.compute(predictions=predictions,refernce=labels)}

In [ ]:
## defining a list of examples

In [ ]:
text_list=["It was not good","Not a fan, don't recomend"]
for text in text_list:
    inputs=tokenizer.encode(text,return_tensors='pt')
    logits=model(inputs).logits
    ## convert logist to labels
    prediction=torch.argmax(logits)
    print(text+"-"+id2label[prediction.tolist()])
    

In [ ]:
## Lora configuration parameters

In [ ]:
pefit_config=LoraConfig(task_type="SEQ_CLS",r=4,lora_alpha=32,  lora_dropout=0.01,target_modules=['q_lin'])

In [ ]:
## hyperparameters

In [ ]:
lr=1e-3
batch_size=4
num_epochs=10
training_args=TrainingArguments(output_dir=model_checkpoint+"lora-text-classification",learning_rate=lr,per_device_train_batch_size=batch_size,num_train_epochs=num_epochs,weight_decay=0.01,evaluation_stratergy="epoch",save_stratergy="epoch",load_best_model_at_end=True,)

In [23]:
 a=[(1, "peter"), (3, "Sam"), (5, "peter")]
 res=[ele for ele in a if ele[1]=="peter"]
 print(res)

[(1, 'peter'), (5, 'peter')]


In [19]:
def get_middle(l1):
    middle_element=[]
    for ele in l1:
        middles=len(ele)//2
        element=ele[middles]
        middle_elements.append(element)
    return middle_elements
nested_tuples = ((1, 2, 3), (4, 5, 6), (7, 8, 9))
result = get_middle(nested_tuples)
print(result)

AttributeError: 'function' object has no attribute 'append'

In [ ]:
## creater Trainer Object

In [ ]:
trainer=Trainer(model=model,args=training_args,train_dataset=tokenize_dataset["train"],eval_dataset=tokenized_dataset["validation"],tokenizer=tokenizer,data_collator=data_collator,compute_metrics=compute_metrics)

In [ ]:
## Train Model

In [ ]:
trainer.train()